In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from numpy import nan

InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.replace(r'^\s*$', nan, regex=True, inplace=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())
sdata.code = sdata.code.fillna('---')
sdata.total = sdata.total.astype(Int32Dtype())
sdata.age = sdata.age.astype(Int32Dtype())
sdata = sdata[sdata.total>=14]
# close the pool of connection
engine.dispose()

True

In [3]:
data = read_excel("./bonscolarisation.xlsx")

In [4]:
data.columns

Index(['case_id', 'code', 'COMMUNE'], dtype='object')

In [6]:
payed = data[data.case_id!="---"]

In [7]:
sdata.case_id.count()
payed.code.count()


57521

22

In [10]:
sp = sdata[sdata.code.isin(payed.code)]
sp.code.count()

22

In [13]:
sp[['case_id','code','commune','actual_departement','first_name','last_name','age']]

,case_id,code,commune,actual_departement,first_name,last_name,age
4501,10223922-83d8-48c7-a2c8-2055633487a7,PAP/DRMS/178442501,port-au-prince,Ouest,Monica,Valhomme,18
4558,105abcfc-0dd3-4a22-ba81-316f03d5ce28,PAP/DRMS/178442502,port-au-prince,Ouest,Angeline,Lochard,22
7503,1b007d76-f670-40b6-9645-fc8aac1b29c1,PAP/DRMS/178443201,port-au-prince,Ouest,Valencia,Melidor,20
13130,2f8de0c5-684c-4d51-a92f-667857462da3,PAP/DRMS/178442503,port-au-prince,Ouest,Rutchelle,St Jean,13
19955,47fe1754-53d7-4224-8529-3faf75af1450,PAP/DRMS/178443202,port-au-prince,Ouest,Benaica,Milus,14
22873,51fd8cb7-feaf-49a4-83c2-09e27177f058,PAP/DRMS/178439424,port-au-prince,Ouest,rose André,Paul Émile,21
23556,545b9463-f50b-45f3-9770-1ad3b3f43a06,PAP/DRMS/178442504,port-au-prince,Ouest,Francesse,Doizan,12
25729,5bd23bc6-690e-4813-a638-0f990439a8d0,PAP/DRMS/178443203,port-au-prince,Ouest,Bedhaissa,Possible,18
26535,5ebde235-8b14-4448-8b3c-ca8cde0489a4,PAP/DRMS/178442505,port-au-prince,Ouest,Scheenyca,Azor,12
49065,aeb56495-8d0e-4670-bf78-3fa2b657fe6a,PAP/DRMS/178443204,port-au-prince,Ouest,Roselande,Jean Baptiste,13


In [16]:
sp.to_excel("./verified_for_bonscolaire.xlsx",index=False,na_rep="")